### JSON  Data
JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. Here is an example:

In [ ]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

JSON is very nearly valid Python code with the exception of its null value null and some other nuances (such as disallowing trailing commas at the end of lists). The basic types are objects (dictionaries), arrays (lists), strings, numbers, Booleans, and nulls. All of the keys in an object must be strings. There are several Python libraries for reading and writing JSON data. 

In [ ]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

```
import json
with open('file.json','r') as f:
    obj = json.load(f)

print(obj)    
```

In [ ]:
asjson = json.dumps(result) #covert Python obj to JSON
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

In [ ]:
import pandas as pd

In [ ]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

,name,age
0,Scott,34
1,Katie,42


In [ ]:
# !cat examples/example.json
!type "examples\example.json" #Windows
# Note it is a JSON list using [ ]

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [ ]:
import pandas as pd

In [ ]:
data = pd.read_json("examples/example.json") #assumes each obj is a row
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [ ]:
import sys
data.to_json(sys.stdout) # by col (default)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [ ]:

data.to_json(sys.stdout, orient="records") # by row

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

In [ ]:
import pandas as pd

### XML and HTML: Web Scraping

In [ ]:
#!conda install lxml beautifulsoup4 html5lib

In [ ]:
tables = pd.read_html("examples/fdic_failed_bank_list.html")
len(tables)


1

In [ ]:
tables

[                             Bank Name             City  ST   CERT  \
 0                          Allied Bank         Mulberry  AR     91   
 1         The Woodbury Banking Company         Woodbury  GA  11297   
 2               First CornerStone Bank  King of Prussia  PA  35312   
 3                   Trust Company Bank          Memphis  TN   9956   
 4           North Milwaukee State Bank        Milwaukee  WI  20364   
 ..                                 ...              ...  ..    ...   
 542                 Superior Bank, FSB         Hinsdale  IL  32646   
 543                Malta National Bank            Malta  OH   6629   
 544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
 545  National State Bank of Metropolis       Metropolis  IL   3815   
 546                   Bank of Honolulu         Honolulu  HI  21029   
 
                    Acquiring Institution        Closing Date  \
 0                           Today's Bank  September 23, 2016   
 1              

In [ ]:
failures = tables[0]


In [ ]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [ ]:
failures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547 entries, 0 to 546
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Bank Name              547 non-null    object
 1   City                   547 non-null    object
 2   ST                     547 non-null    object
 3   CERT                   547 non-null    int64 
 4   Acquiring Institution  547 non-null    object
 5   Closing Date           547 non-null    object
 6   Updated Date           547 non-null    object
dtypes: int64(1), object(6)
memory usage: 30.0+ KB


In [ ]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps


0     2016-09-23
1     2016-08-19
2     2016-05-06
3     2016-04-29
4     2016-03-11
         ...    
542   2001-07-27
543   2001-05-03
544   2001-02-02
545   2000-12-14
546   2000-10-13
Name: Closing Date, Length: 547, dtype: datetime64[ns]

In [ ]:
close_timestamps.dt.year.value_counts()

Closing Date
2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, dtype: int64

### Parsing XML with lxml.objectify

In [ ]:
from lxml import objectify

path = "datasets/mta_perf/Performance_MNR.xml"
with open(path) as f:
    parsed = objectify.parse(f)
root = parsed.getroot()
root

<Element PERFORMANCE at 0x24ef14489c0>

`root.INDICATOR` returns a generator yielding each <INDICATOR> XML element. For each record, we can populate a dictionary of tag names (like YTD_ACTUAL) to data values (excluding a few tags) by running the following code:

In [ ]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


In [ ]:
perf2 = pd.read_xml(path)
perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80


## 6.3 Interacting with Web APIs
 Install in the venv 
 
 ```
 conda install requests
 ```

Example: To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP request using the add-on requests library:

<font color='red'>Note</font>

:::{.callout-note}
Note that there are five types of callouts, including: 
`note`, `tip`, `warning`, `caution`, and `important`.


In [ ]:
import requests
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.raise_for_status()
resp


<Response [200]>

It's a good practice to always call raise_for_status after using requests.get to check for HTTP errors.

The response object’s json method will return a Python object containing the parsed JSON data as a dictionary or list (depending on what JSON is returned):

In [ ]:
data = resp.json()
data

[{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/55607',
  'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
  'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/55607/labels{/name}',
  'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/55607/comments',
  'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/55607/events',
  'html_url': 'https://github.com/pandas-dev/pandas/pull/55607',
  'id': 1954538147,
  'node_id': 'PR_kwDOAA0YD85dZUhv',
  'number': 55607,
  'title': 'REF: initialize Series._name for class instead of after _from_mgr',
  'user': {'login': 'jorisvandenbossche',
   'id': 1020496,
   'node_id': 'MDQ6VXNlcjEwMjA0OTY=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/1020496?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/jorisvandenbossche',
   'html_url': 'https://github.com/jorisvandenbossche',
   'followers_url': 'https://api.github.com/users/jorisvande

In [ ]:
data[0]["title"]

'REF: initialize Series._name for class instead of after _from_mgr'

In [ ]:
issues = pd.DataFrame(data, columns=["number", "title",
                                     "labels", "state"])
issues

,number,title,labels,state
0,55607,REF: initialize Series._name for class instead...,[],open
1,55606,BUG: regression in read_parquet that raises a ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,55605,BUG: hash_array does not produce deterministic...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,55604,BUG: using iloc/loc to set a nullable int type...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,55603,TST: move misplaced tests,[],open
5,55601,BUG(?) `Index.__getitem__` with 0D numpy array...,"[{'id': 2822098, 'node_id': 'MDU6TGFiZWwyODIyM...",open
6,55598,ENH: Allow rank to return int64 for numpy type...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
7,55594,BLD: Allow building with NumPy nightlies and u...,"[{'id': 129350, 'node_id': 'MDU6TGFiZWwxMjkzNT...",open
8,55592,ENH: functions filter_columns and filter_rows ...,[],open
9,55591,DEPR: fix stacklevel for DataFrame(mgr) deprec...,"[{'id': 49094459, 'node_id': 'MDU6TGFiZWw0OTA5...",open
